In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import pandas as pd
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratJday1
    +subjects.sd.ratKday1
    + subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratRday2
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2

    + subjects.nsd.ratJday2
    + subjects.nsd.ratKday2
    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratRday1
    + subjects.nsd.ratUday2
    + subjects.nsd.ratVday1
)

### Plot ripples histogram for a single session

In [ ]:
from neuropy.plotting import plot_epochs
from scipy.stats import binned_statistic

sessions = subjects.sd.ratUday1

fig = Fig(grid=(2, 2))
for sub, sess in enumerate(sessions):
    ax = plt.subplot(fig.gs[sub])
    starts = sess.ripple.starts
    durations = sess.ripple.durations
    bins = np.arange(0, sess.eegfile.duration, 600)
    # hist_rpl = np.histogram(starts,bins=bins)[0]
    hist_rpl = binned_statistic(starts, durations, bins=bins, statistic="sum")[0]
    plot_epochs(ax, sess.paradigm)
    ax.plot(bins[:-1] + 300, hist_rpl / 600, "k")
    ax.set_xlabel("Time")
    ax.set_ylabel("Ripple rate")
    print(sess.ripple.metadata)


### Ripple rate first hour **between** SD and NSD (or only restrict to 1h NREM sleep of NSD)
- One possiblity of faster decay of replay during sleep deprivation could be reduced ripple rate at the begining of POST. This may suggest that in order for cells to replay they need sufficient number/amount of exicitation to last for certain duration of time, so a lower ripple rate could interrupt this requirement.

In [ ]:
rpl_rate = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    w = 2 * 3600 # window size
    rpl_rate_sub = sess.ripple.time_slice(post[0], post[0] + w).n_epochs / w
    rpl_rate.append(
        pd.DataFrame({"sub": sess.sub_name, "rpl_rate": [rpl_rate_sub], "grp": sess.tag})
    )

rpl_rate = pd.concat(rpl_rate,ignore_index=True)


In [ ]:

_,ax = plt.subplots()
import seaborn as sns

sns.lineplot(data=rpl_rate,x='grp',y='rpl_rate',hue='sub',marker='o')

### Ripple rate first 5 hour **within** SD and NSD across Zt times


In [ ]:
ripple_rate_df = pd.DataFrame()
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    period = [post[0], post[0] + 9* 3600]
    rpls = sess.ripple.starts

    binsz = 3600
    n_bins = np.diff(period)/ binsz
    sd_bin = np.arange(n_bins + 1) * binsz + period[0]
    hist_rpl = np.histogram(rpls, bins=sd_bin)[0]
    # hist_rpl = hist_rpl / np.sum(hist_rpl)

    ripple_rate_df = ripple_rate_df.append(
        pd.DataFrame(
            {
                "rate": hist_rpl / binsz,
                "Zt": (sd_bin[:-1] - sd_bin[0]) / 3600 +1,
                "sub": sub,
                "grp": sess.tag,
            }
        ),
        ignore_index=True,
    )

# subjects.GroupData().save(ripple_rate_df,'ripple_rate')

In [ ]:
import seaborn as sns
from neuropy.plotting import Fig

fig = Fig(grid=(5, 4))

for i, grp in enumerate(["sd",'nsd']):
    df = ripple_rate_df[ripple_rate_df["grp"] == grp]
    ax = fig.subplot(fig.gs[i])
    sns.lineplot(
        data=df,
        x="Zt",
        y="rate",
        ax=ax,
        hue="sub",
        legend=False,
        palette=["#BDBDBD"] * 8,
        # ls="--",
    )
    sns.lineplot(
        data=df,
        x="Zt",
        y="rate",
        estimator=np.median,
        ax=ax,
        legend=False,
        color='k',
        ls="--",
        ci=None,
        lw=2,
    )
    # ax.set_yscale('log')
    ax.set_ylim([0.08, 0.65])
    ax.set_xticks(np.arange(1, 10, 2))

# figure.savefig(subjects.figpath_sd/'ripple_rate')

### Inter-ripple duration (Ripple ISI)


In [ ]:
dur_df = pd.DataFrame()
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    zts =np.array([0,4,5]) # np.arange(0, 9, 4)
    t_starts = zts * 3600 + post[0]

    acg_zt = []
    firing_neurons = []
    for i, t in enumerate(t_starts):
        starts = sess.ripple.time_slice(t, t + 3600).starts
        dur_df = dur_df.append(
            pd.DataFrame(
                {"isi": np.diff(starts), "Zt": f"Zt{zts[i]+1}", "grp": sess.tag}
            ),
            ignore_index=True,
        )


In [ ]:
%matplotlib widget
import seaborn as sns
from neuropy import plotting

figure = plotting.Fig()
fig, gs = figure.draw(grid=(5, 4))
bins = [np.arange(2,50,0.1),np.arange(0.5,50,0.1)]
for i, g in enumerate(["sd", "nsd"]):
    df = dur_df[dur_df['grp']==g]
    ax = plt.subplot(gs[i])
    sns.histplot(
        data=df,
        x="isi",
        hue="Zt",
        ax=ax,
        stat='probability',
        bins= bins[i],
        element='poly',
        common_bins=True,
        common_norm=False,
        fill=False,
        legend=None,
        cumulative=True,
        palette='Set2'
    )
    ax.set_xscale('log')
    # ax.set_yscale('log')
    # ax.grid()

figure.savefig(subjects.figpath_sd/'ripple_isi')

### Total/mean ripple duration from first hour to last hour SD

In [ ]:
rpl_duration = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    period = np.arange(9) * 3600 + post[0]

    for i, t in enumerate(period):

        rpl_duration.append(
            pd.DataFrame(
                {
                    "Zt": f"{i+1}",
                    "rpl_duration": [sess.ripple.time_slice(t, t + 3600).durations.sum()],
                    "grp": sess.tag,
                    "sub": sub,
                }
            ),
        )

rpl_duration = pd.concat(rpl_duration,ignore_index=True)

# subjects.GroupData().save(rpl_duration, "ripple_total_duration")


In [ ]:
import seaborn as sns
from neuropy import plotting

fig = plotting.Fig(grid=(4,3))

colors =[subjects.sd.color(1.5),subjects.nsd.color(1.5)]
n_subs=[8,8]
for i, grp in enumerate(["SD", "NSD"]):
    data = rpl_duration[rpl_duration["grp"] == grp]
    ax = fig.subplot(fig.gs[i])
    sns.lineplot(
        data=data,
        x="Zt",
        y="rpl_duration",
        hue="sub",
        marker="o",
        palette=[colors[i]]* n_subs[i],
        ax=ax,
        legend=None,
        ci=None,
    )
    ax.legend('')
    # ax.set_xlim([-1,4])
    ax.set_ylim([30,450])


# g=sns.relplot(
#     data=rpl_duration,
#     x="epoch",
#     y="rpl_duration",
#     hue="sub",
#     col="grp",
#     # row='rpl_duration',
#     kind="line",
#     marker="o",
#     palette=['gray']*len(sessions),
# )

# [ax.set_xlim([-1,2]) for ax in g.axes.reshape(-1)]
# figure.savefig(subjects.figpath_sd/'ripple_total_duration_zt1_vs_zt5')